In [161]:
import numpy as np
#Define the environment
class SimuEnv:
    #The defined model as class parameter
    # P[s][a][s']
    transition_probabilities = [
        [[0.7, 0.3, 0.0], [1.0, 0.0, 0.0], [0.8, 0.2, 0.0]], # in s0, if action a0 then proba 0.7 to state s0 and 0.3 to state s1, etc.
        [[0.0, 1.0, 0.0], None, [0.0, 0.0, 1.0]],
        [None, [0.8, 0.1, 0.1], None],
        ]
    # R[s][a][s']
    rewards = [
        [[+10, 0, 0], [0, 0, 0], [0, 0, 0]],
        [[0, 0, 0], [0, 0, 0], [0, 0, -50]],
        [[0, 0, 0], [+40, 0, 0], [0, 0, 0]],
        ]

    possible_actions = [[0, 1, 2], [0, 2], [1]]
    n_states = 3
    n_actions = 3
    def __init__(self, start_state = 0, discount_rate = 1):
        self.start_state = start_state
        self.total_reward = 0
        self.discount_rate = discount_rate
        self.reset()
    
    def reset(self):
        self.current_state = self.start_state
        self.total_reward = 0
        return self.start_state, 0, False, ""
    
    def step(self, action):
        if action not in SimuEnv.possible_actions[self.current_state]:
            return 0, 0, True, "Illegal action for state %d." % (self.current_state)
        trans_prop = SimuEnv.transition_probabilities[self.current_state][action]
        next_state = np.random.choice(3, 1, p = trans_prop)[0]
        reward = SimuEnv.rewards[self.current_state][action][next_state]
        self.current_state = next_state
        self.total_reward = self.discount_rate * self.total_reward + reward
        return next_state, reward, False, ""

def policy_try(state):
    return [0, 2, 1][state]

def policy_safe(state):
    return [0, 0, 1][state]

def simulation(policy):
    env = SimuEnv(discount_rate=0.95)
    obs = env.reset()
    op_stack = []
    for i in range(10000):
        state = obs[0]
        action = policy(state)
        obs = env.step(action)
        op_stack.append((state, action, obs[1], obs[0]))
    
    #print (op_stack)
    return op_stack, env.total_reward


In [162]:
def get_avg_reward(policy, n_iteration):
    avg = 0
    for iter in range(n_iteration):
        op_stack, total_reward = simulation(policy)
        avg = (total_reward + iter * avg) / (iter + 1)
    return avg

In [163]:
print(get_avg_reward(policy_try, 10))


33.593528520829864


In [157]:
"""
Q Value Iteration......
Bellman's Equation
Q(s, a) = sum (P(s,a,s') * (R(s, a, s') + gama * [max(a') s.t. Q(s', a')] ))
"""
def Qiter(model, gama, n_iter):
    Q = np.zeros([model.n_states, model.n_actions])
    tp = model.transition_probabilities
    r = model.rewards
    for iter in range(n_iter):
        for state in range(model.n_states):
            for action in model.possible_actions[state]:
                Q[state][action] = sum([tp[state][action][s_state] *\
                                        (r[state][action][s_state] +\
                                         gama *\
                                         max([Q[s_state][action] for action in model.possible_actions[s_state]])) 
                                        for s_state in range(model.n_states)])
    return Q

In [164]:
Q = Qiter(SimuEnv, 1, 20000)

In [165]:
Q


array([[30920.73009586, 30920.73009586, 30916.78464131],
       [30901.00282313,     0.        , 30902.54827768],
       [    0.        , 30954.09373222,     0.        ]])

In [160]:
np.random.choice(range(3), 10, p = [0.5, 0.5, 0])

array([0, 1, 0, 0, 0, 1, 0, 1, 1, 1])

In [169]:
"""
Q learning with TD(0) and sigma-greedy:
Q(s, a) = (1 - a)Q(s, a) + a(r(s,a,s') + gama * max(a') [Q(s', a')])
"""

def Qlearning(model, gama, alpha, n_iter, sigma, decay_rate):
    n = np.full((model.n_states, model.n_actions), 0)
    Q = np.full((model.n_states, model.n_actions), 0.)
    Q[1, 1] = -np.inf
    Q[2, 0], Q[2, 2] = -np.inf, -np.inf
    env = model()
    state = 0
    pa = model.possible_actions
    for iteration in range(n_iter):
        action = np.random.choice(pa[state])
        obs = env.step(action)
        s_state, s_reward, done, info = obs
        n[state][action] += 1
        Q[state][action] = (1 - alpha)*Q[state][action] + alpha * (s_reward + gama * max([Q[s_state][s_action] for
                                                                                         s_action in pa[s_state]]))
        alpha = alpha / (1 + iteration * decay_rate / 10.0)
    return Q
        

In [175]:
Q

array([[ 0.0004796 ,  0.15982627, -0.27980236],
       [ 0.        ,        -inf,  0.        ],
       [       -inf,  0.        ,        -inf]])